# Harmonic oscillator

Numerical solution of simple harmonic oscillator, and double spring oscillator.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
from dataclasses import dataclass
import os

plt.style.use("bmh")
#plt.style.use("dark_background")

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100
%config InlineBackend.figure_format = 'retina'
%config InteractiveShellApp.matplotlib = "inline"


In [ ]:
print("Making sure current working directory is in this directory")
try:
    os.chdir("ode/harmonic_oscillator1")
except Exception as e:
    print("Current directory:", os.getcwd())
    print(e)
    print("New directory", os.getcwd())

In [ ]:
!pwd
%load_ext autoreload
%autoreload 2
    
from ode.harmonic_oscillator1.solvers import \
    SolverResult, \
    BackwardEulerSolver, \
    ForwardEulerSolver, \
    SemiImplicitEulerSolver1, \
    SemiImplicitEulerSolver2, \
    SemiImplicitEulerSolverAvg, \
    VodeSolver, \
    VelocityVerletSolver

from ode.harmonic_oscillator1.equations import \
    DoubleSpring1, \
    HarmonicOscillator1

# Simple 1D spring without dampening

```
/|
/|--vvvv----- m
/|
 ----------> x axis
```

- k = spring_constant
- L = resting_length
- F = ma

$$ F = -k(x - L) = ma $$

$$=>   m dv/dt = -k(x - L)$$

### System of equations:

$$
\begin{align}
dx/dt &= v                 =   0        +    1 * v \\
dv/dt &= -k/m (x - L)      = -k/m x     +    0 * v    + kL/m
\end{align}
$$

Substitute $x = x- L$ to remove the constant term:

$$
\begin{align}
dx/dt &= v \\
dv/dt &= -k/m x
\end{align}
$$

Matrix form: Let the vector $\vec x$ contain the entire state of the system. 

$$\vec x = \begin{bmatrix}x\\ v\end{bmatrix}$$

The equation above becomes the following:

$$\frac{d\vec x}{dt} = \begin{bmatrix}0 & 1\\ -k/m & 0\end{bmatrix} \, \vec x = f(t, x)$$

Some methods additionally require a Jacobian matrix, so we compute that as well:

$$
J = J(t, x) = \frac{df}{dx}= \begin{bmatrix}0 & 1\\ -k/m & 0\end{bmatrix}
$$

Furthermore, symplectic integrators require splitting the state vector $\mathbf x$ into generalized position and momentum coordinates:

$$\mathbf x = \begin{bmatrix}\mathbf q\\ \mathbf p\end{bmatrix},$$

and splitting $\mathbf f$ into $f_q(t, q)$

$$\mathbf f(\mathbf t, \mathbf q, \mathbf p) = \begin{bmatrix}\mathbf f_q(t, p)\\ \mathbf f_p(t, q)\end{bmatrix},$$

## Solver comparison functions

In [ ]:
def plot_solver_results(solver, dts, r0, t_start=0.0, t_end=0.05):
    compute_times = []
    #plt.figure(figsize=(0.4,0.2))
    for dt in dts:
        tt = np.arange(t_start, t_end, dt)
        result = solver.solve(tt, r0)
        compute_times.append(result.compute_time)
        positions = result.xs[0,:]
        plt.plot(tt*1000, positions*100)
        plt.title(f"Simple harmonic oscillator: {solver.name}")

    legends = [f"dt={dt}, compute={compute:.2f}" for (dt, compute) in zip(dts, compute_times)]
    plt.legend(legends)
    plt.xlabel("milliseconds")
    plt.ylabel("cm displacement")
    plt.show()

def plot_solver_comparison(equation_name, solvers, t_start=0.0, t_end=0.074):
    plt.figure(figsize=(16, 6))
    compute_times = []
    for (solver, dt, r0) in solvers:
        tt = np.arange(t_start, t_end, dt)
        result = solver.solve(tt, r0)
        compute_times.append(result.compute_time)
        positions = result.xs[0,:]
        times = result.ts
        plt.plot(times*1000, positions*100)

    plt.title(f"{equation_name}")
    legends = [f"{solver.name}, dt={dt:.2e}, compute={compute:.2f}" for ((solver, dt), compute) in zip(solvers, compute_times)]
    plt.legend(legends)
    plt.xlabel("milliseconds")
    plt.ylabel("cm displacement")
    plt.show()



## The unreasonable accuracy of VODE

VODE is just extremely good even for large time steps.

In [ ]:



equation = HarmonicOscillator1()
f, J, fq, fp = equation.f, equation.J, equation.fq, equation.fp
r0 = np.array([1.0, 0.0])
q0 = r0[0]
p0 = r0[1]

dts = [0.0005, 0.0025]
plot_solver_results(ForwardEulerSolver(f), dts, r0)
plot_solver_results(BackwardEulerSolver(f, J), dts, r0)
plot_solver_results(SemiImplicitEulerSolver1(fq, fp), dts, r0)
plot_solver_results(VodeSolver(f, J), dts, r0)

# Double springs

```

/|     k1      m1       k2       m2
/|---vvvvv----###----vvvvvv-----###
/|
```

In [ ]:
def plot_double_spring_results(solver, dt, r0=None, q0=None, p0=None, t_start=0.0, t_end=0.025):
    t_start = 0.0
    compute_times = []
    tt = np.arange(t_start, t_end, dt)
    if r0 is not None:
        result = solver.solve(tt, r0)
    else:
        result = solver.solve(tt, q0, p0)
    positions_mass0 = result.xs[0,:]
    positions_mass1 = result.xs[1,:]
    #plt.plot(tt*1000, (positions_mass0)*100)
    plt.plot(tt*1000, (positions_mass0)*100)
    plt.plot(tt*1000, (positions_mass1)*100)
    plt.title(f"Double spring: {solver.name}")

    legends = ["Mass 1", "Mass 2"]
    plt.legend(legends)
    plt.xlabel("milliseconds")
    plt.ylabel("cm displacement")
    plt.show()

equation = DoubleSpring1(k0=5000.0, m0=0.2, k1=4000.0, m1=0.1, L0=0.1, L1=0.2)

f, J, fq, fp = equation.f, equation.J, equation.fq, equation.fp
dt = 0.0001

r0 = np.array([0.06, 0.21, 0.0, 0.0])
q0 = r0[:2]
p0 = r0[2:]
plot_double_spring_results(VodeSolver(f, J), dt, r0=r0)
plot_double_spring_results(VelocityVerletSolver(fq, fp), dt, q0=q0, p0=p0)

plot_double_spring_results(ForwardEulerSolver(f), dt, r0=r0)
plot_double_spring_results(SemiImplicitEulerSolver2(fq, fp), dt, q0=q0, p0=p0)
#plot_double_spring_results(BackwardEulerSolver(f, J), dts, r0)
#plot_double_spring_results(VodeSolver(f, J), dt, r0=r0)

# Comparing the accuracy vs VODE

Plotting function

In [ ]:
def plot_double_spring_solver_comparison(results: SolverResult):
    plt.figure(figsize=(14,6))
    for result in results:
        plt.plot(result.ts, result.xs[0,:], linewidth=1)
    plt.legend([result.solver_name for result in results])
    plt.ylabel('displacement')
    plt.xlabel('time')


Define the equation and initial conditions

In [ ]:
f, J, fq, fp = equation.f, equation.J, equation.fq, equation.fp

r0 = np.array([0.06, 0.21, 0.0, 0.0])
q0 = r0[:2]
p0 = r0[2:]

Solve the equation using multiple solvers and compare the results

In [ ]:
dt_precise = 0.00001
exact_solution = VodeSolver(f, J).solve(tt_precise, r0)

## Velocity verlet solver for double spring

In [ ]:
dt = 0.001
tt = np.arange(0, 0.2, dt)
tt_precise = np.arange(0, 0.2, dt)
plot_double_spring_solver_comparison([
    VelocityVerletSolver(fq, fp).solve(tt, q0, p0),
    exact_solution,
])

## Semi implicit solver for double spring

In [ ]:
dt = 0.001
tt = np.arange(0, 0.2, dt)
tt_precise = np.arange(0, 0.2, dt)
plot_double_spring_solver_comparison([
    SemiImplicitEulerSolver2(fq, fp).solve(tt, q0, p0),
    exact_solution,
])

## Explicit euler for double spring

In [ ]:
dt = 0.0005
tt = np.arange(0, 0.2, dt)
tt_precise = np.arange(0, 0.2, dt)
plot_double_spring_solver_comparison([
    ForwardEulerSolver(f).solve(tt, r0),
    exact_solution,
])